# Advanced Financial Performance Analysis — Uploaded Fundamentals Dataset

This notebook uses the uploaded `fundamentals_dataset.csv` (long-form indicators). It pivots indicators to a wide table per company-period, computes financial ratios, performs peer benchmarking, time-series analysis, forecasting, and scenario analysis.

**Notes:** The notebook auto-detects commonly used indicators (Revenue, Net Income, Total Assets, Total Liabilities, Shareholders' Equity, Gross Profit) from the `indicator` column. If an indicator is missing, ratio calculations skip gracefully.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
sns.set_style('whitegrid')
%matplotlib inline


In [ ]:
# Load the uploaded dataset
fn = '/mnt/data/fundamentals_dataset.csv'
df_long = pd.read_csv(fn)
print('Rows:', len(df_long))
print('Columns:', df_long.columns.tolist())
# show top indicators
print('\nSample rows:')
print(df_long.head(6).to_string(index=False))


In [ ]:
# Clean amount: remove commas, parentheses, handle negatives
def parse_amount(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    # negative in parentheses
    if s.startswith('(') and s.endswith(')'):
        s = '-' + s[1:-1]
    s = s.replace(',','').replace('$','')
    try:
        return float(s)
    except Exception:
        return np.nan

df_long['amount_clean'] = df_long['amount'].apply(parse_amount)
# normalize period to quarter end date
# period values like '2014 Q1'
def period_to_date(p):
    try:
        y,q = p.split()
        year = int(y)
        quarter = int(q.replace('Q',''))
        month = quarter*3
        return pd.Timestamp(year=year, month=month, day=1) + pd.offsets.MonthEnd(0)
    except Exception:
        return pd.NaT

if 'period' in df_long.columns:
    df_long['period_end'] = df_long['period'].astype(str).map(period_to_date)
else:
    df_long['period_end'] = pd.NaT

print('\nParsed period samples:')
print(df_long[['period','period_end']].drop_duplicates().head())


In [ ]:
# Pivot to wide format: each indicator becomes a column
# first pick a shorter name for indicators by removing excessive spacing

df_long['indicator_clean'] = df_long['indicator'].str.strip()
# pivot
pivot = df_long.pivot_table(index=['tickers','company','period','period_end'], columns='indicator_clean', values='amount_clean', aggfunc='first').reset_index()
print('Pivot shape:', pivot.shape)
print('Available indicators (columns):')
print([c for c in pivot.columns if c not in ['tickers','company','period','period_end']][:50])


In [ ]:
# Define common indicator name matches (try to find columns corresponding to revenue, net income, assets, liabilities, equity)
cols = pivot.columns.tolist()
col_lower = {c:c.lower() for c in cols}
# helper to find best match

def find_col(keywords):
    for k in keywords:
        for c in cols:
            if k.lower() in str(c).lower():
                return c
    return None

revenue_col = find_col(['revenue','total revenue','operating revenue','sales'])
net_income_col = find_col(['net income','netincome','profit (loss) from continuing operations','net loss','net profit'])
assets_col = find_col(['total assets','assets'])
liabilities_col = find_col(['total liabilities','liabilities'])
equity_col = find_col(['shareholder','stockholders equity','total equity','shareholders\' equity','equity'])

print('Detected columns:')
print('Revenue:', revenue_col)
print('Net income:', net_income_col)
print('Total assets:', assets_col)
print('Total liabilities:', liabilities_col)
print('Equity:', equity_col)


In [ ]:
# Compute financial ratios if columns are available
pf = pivot.copy()
if revenue_col:
    pf['revenue'] = pf[revenue_col]
else:
    pf['revenue'] = np.nan
if net_income_col:
    pf['net_income'] = pf[net_income_col]
else:
    pf['net_income'] = np.nan
if assets_col:
    pf['total_assets'] = pf[assets_col]
else:
    pf['total_assets'] = np.nan
if liabilities_col:
    pf['total_liabilities'] = pf[liabilities_col]
else:
    pf['total_liabilities'] = np.nan
if equity_col:
    pf['equity'] = pf[equity_col]
else:
    # try estimate
    pf['equity'] = pf['total_assets'] - pf['total_liabilities']

# ratios
pf['net_margin'] = pf['net_income'] / pf['revenue']
pf['roa'] = pf['net_income'] / pf['total_assets']
pf['de_ratio'] = pf['total_liabilities'] / pf['equity']
pf['revenue'] = pf['revenue'].astype(float)

print('Sample KPIs:')
print(pf[['tickers','company','period','revenue','net_income','net_margin','roa','de_ratio']].head())


## KPI summary (latest period per company)

In [ ]:
# latest period per ticker
latest = pf.sort_values('period_end').groupby('tickers').tail(1).set_index('tickers')
kpi_cols = ['company','period','period_end','revenue','net_income','net_margin','roa','de_ratio']
latest_kpis = latest[kpi_cols].sort_values('revenue', ascending=False)
latest_kpis.head(20)
# export
latest_kpis.to_csv('/mnt/data/latest_kpis_from_uploaded.csv')
print('Exported latest_kpis_from_uploaded.csv')


## Peer benchmarking — choose tickers and compare ratios

In [ ]:
# Example peers: top tickers by revenue
top10 = latest_kpis.sort_values('revenue', ascending=False).head(10)
peers = top10.index.tolist()
print('Top peers (by revenue):', peers)

peer_df = latest_kpis.loc[peers]
plt.figure(figsize=(10,5))
sns.barplot(x=peer_df.index, y=peer_df['net_margin'])
plt.title('Peer Net Margin (latest)')
plt.ylabel('Net Margin')
plt.show()


## Time-series analysis & forecasting for a selected ticker

In [ ]:
# select ticker
sel = peers[0] if peers else None
print('Selected ticker for TS analysis:', sel)
if sel:
    comp = pf[pf['tickers']==sel].sort_values('period_end').set_index('period_end')
    ts = comp['revenue'].dropna()
    print('Time points:', len(ts))
    plt.figure(figsize=(10,4))
    ts.plot(marker='o'); plt.title(f'{sel} Revenue over time'); plt.show()
    # QoQ growth
    ts_q = ts.resample('Q').sum()
    ts_q_growth = ts_q.pct_change()
    print(ts_q.tail())
    # Forecast if enough points
    if len(ts_q.dropna()) >= 8:
        try:
            model = ARIMA(ts_q, order=(1,1,1)).fit()
            fc = model.get_forecast(steps=4)
            pred = fc.predicted_mean
            ci = fc.conf_int()
            future_idx = pd.date_range(start=ts_q.index[-1] + pd.offsets.QuarterBegin(), periods=4, freq='Q')
            plt.figure(figsize=(10,4))
            plt.plot(ts_q.index, ts_q, label='observed')
            plt.plot(future_idx, pred, marker='o', label='forecast')
            plt.fill_between(future_idx, ci.iloc[:,0], ci.iloc[:,1], alpha=0.2)
            plt.legend(); plt.title(f'{sel} Revenue Forecast (4 Qtrs)'); plt.show()
        except Exception as e:
            print('ARIMA failed:', e)
else:
    print('No ticker selected or no data available')


## Scenario analysis (What-if): revenue shock impact on net income and ratios

In [ ]:
# compute impact for selected ticker using historic net margin
if sel and sel in latest_kpis.index:
    base = latest_kpis.loc[sel]
    shock = -0.1  # -10% revenue
    hist = pf[pf['tickers']==sel]
    hist_net_margin = hist['net_income'].sum() / max(hist['revenue'].sum(),1)
    rev0 = base['revenue']
    net0 = base['net_income']
    rev_shock = rev0 * (1+shock)
    net_shock = rev_shock * hist_net_margin
    print(f"{sel} baseline revenue={rev0:,.0f}, net_income={net0:,.0f}")
    print(f"After {shock*100:.0f}% shock -> revenue={rev_shock:,.0f}, projected net_income={net_shock:,.0f}")
else:
    print('No data for scenario analysis')


## Additional analytics: profitability scoring & export
Compute a simple score combining growth and margins to identify attractive companies.

In [ ]:
# compute revenue growth over last 4 quarters where possible
pf_sorted = pf.sort_values(['tickers','period_end'])
# take last 4 quarters revenue sum and previous 4 quarters
scores = []
for t, g in pf_sorted.groupby('tickers'):
    gq = g.set_index('period_end').resample('Q').sum()
    if len(gq) >= 8:
        last4 = gq['revenue'].iloc[-4:].sum()
        prev4 = gq['revenue'].iloc[-8:-4].sum()
        growth = (last4 - prev4)/prev4 if prev4>0 else np.nan
    else:
        growth = np.nan
    latest_row = g.iloc[-1]
    score = np.nanmean([latest_row.get('net_margin', np.nan), latest_row.get('roa', np.nan), growth])
    scores.append({'tickers':t,'company':latest_row.get('company'),'score':score,'growth':growth,'net_margin':latest_row.get('net_margin')})

score_df = pd.DataFrame(scores).sort_values('score', ascending=False).reset_index(drop=True)
score_df.head(20)
# export
score_df.to_csv('/mnt/data/company_scores.csv', index=False)
print('Exported /mnt/data/company_scores.csv')


## Done — files exported:
- `/mnt/data/latest_kpis_from_uploaded.csv` (latest KPI table)
- `/mnt/data/company_scores.csv` (profitability & growth scores)

You can download these files from the session. Inspect column names if you want to rename detections (revenue_col, net_income_col etc.) for better matching.